In [6]:
import os 
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from pydantic import BaseModel,Field
from typing import Optional,Literal
from typing import TypedDict
load_dotenv()



GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


In [2]:
Model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",api_key=GOOGLE_API_KEY)
Model.invoke('What is 2+2?')

AIMessage(content='2+2 = 4', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--52d20312-1179-4576-b545-29fe44bb36e1-0', usage_metadata={'input_tokens': 8, 'output_tokens': 6, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}})

In [7]:
# lets make a review replier which will check review is postive
# or negative,if its postive then simply thanks to the customer
# if its negative then we will check the problem , user tone like 
# anger, calm etc and urgency type .
# then based on it we will assist the user. 

In [24]:
class SentimentSchema(BaseModel):
    sentiment  : Literal['positive','negative'] = Field(description="sentiment of the review")
    

In [25]:
Model_01 = Model.with_structured_output(SentimentSchema)

In [26]:
review = "this phone is pathetic"
prompt = "what is review of the sentiment - this software is to good"
Model_01.invoke(prompt).sentiment

'positive'

In [34]:
class reviewState(TypedDict):
    review:str
    sentiment:Literal['positive','negative']
    daignosis:dict
    response:str



In [ ]:
def find_sentiment(state:reviewState):
    review = state['review']
    prompt = f"for the following review find the sentiment \n {review}"
    result = Model_01.invoke(prompt)
    return {'sentiment':result}

def check_sentiment(state:reviewState)->Literal['positive_response','daignose']:
    if state['sentiment'] == 'positive':
        return 'positive_response'
    else:
        'daignose'

In [ ]:
from langgraph.graph import StateGraph,START,END

graph = StateGraph(reviewState)
graph.add_node("find_sentiment",find_sentiment)
graph.add_conditional_edges('find_sentiment',"check_sentiment")
graph.add_node("positive_response",positive_response)
graph.add_node("daignose",daignose)
graph.add_edge(START,'find_sentiment')
graph.add_edge('find_sentiment',END)

workflow = graph.compile()

In [33]:
initial_state= {"review":"this phone is very good user experiance i appricate it"}
result=workflow.invoke(initial_state)
print(result)

{'review': 'this phone is very good user experiance i appricate it', 'sentiment': SentimentSchema(sentiment='positive')}
